# Minimum Buck Converter Voltage Conversion Ratio Calculation
**Author:** Ian Eykamp, **Date:** July 11, 2023

### Purpose
To determine whether a 1kV to 12V buck converter is possible.

### Methodology
According to this [document](https://www.ti.com/lit/an/slyt293/slyt293.pdf), the lower bound on the buck converter's voltage conversion ratio is driven by the minimum on time $T_{on, min}$, which has to do with the controller's ability to react in time to a quickly rising current. This will affect the controller's minimum possible duty cycle.

For a given $T_{on, min}$, we can have a duty cycle of $\frac{T_{on, min}}{T_s}$, where $T_s = \frac{1}{F_s}$ is the switching period.

This can be confirmed by looking at the inductor's curent waveform. On the inductor rising edge, we have $\frac{dI_{rise}}{dt} = \frac{(V_g - V_{out})}{L}$ lasting $T_{on, min}$, and on the falling edge, we have $\frac{dI_{fall}}{dt} = -\frac{V_{out}}{L}$ lasting $T_s - T_{on, min}$. Solving for $\frac{dI_{rise}}{dt} \cdot T_{on, min} = -\frac{dI_{fall}}{dt} \cdot (T_s - T_{on, min})$ gives

$\frac{(V_g - V_{out})}{L} \cdot T_{on, min} = \frac{V_{out}}{L} \cdot (T_s - T_{on, min})$

$(V_g - V_{out}) \cdot T_{on, min} = V_{out} \cdot (T_s - T_{on, min})$

$V_g \cdot T_{on, min} - V_{out} \cdot T_{on, min} = V_{out} \cdot T_s - V_{out} \cdot T_{on, min}$

$V_g \cdot T_{on, min} = V_{out} \cdot T_s$

$\frac{T_{on, min}}{T_s} = \frac{V_{out}}{V_g}$, as expected.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from UliEngineering.EngineerIO import format_value
from si_prefix import si_format
import plecs_helper as helper
%matplotlib
%matplotlib inline

# Imports and setup
from pint import UnitRegistry

# pandas display using scientific notation
# pd.set_option('display.float_format', lambda x: f'{x:.3e}')

# use pint
units = UnitRegistry()
units.default_format = "~P.2f"

Using matplotlib backend: TkAgg


In [7]:
Fs = 10 * units.kilohertz
Ts = 1 / Fs
Tmin_on = 600 * units.nanosecond
Dmin = Tmin_on / Ts
print(f"Minimum duty cycle given Tmin_on = {Tmin_on.to_compact(units.second)} is D = {np.round(Dmin.to('').magnitude, 4)}")


Vg = 1000 * units.volt
Vout = 12 * units.volt
Dwant = Vout / Vg

print(f"Desired duty cycle given Vg = {Vg.to_compact(units.volt)} and is Vout = {Vout.to_compact(units.volt)} is D = {np.round(Dwant.to('').magnitude, 4)}")

Vdiode = 1 * units.volt
Rds1 = 100 * units.milliohm
RL = 200 * units.milliohm
Iout_min = 1 * units.amp
Vout_min = Tmin_on * Fs * (Vg - (Iout_min * Rds1 - Vdiode)) - (Iout_min * (RL) + Vdiode)
# Vout_min = Tmin_on * Fs * Vg

print(f"Minimum output voltage Vout_min = {Vout_min.to_compact(units.volt)}")

Minimum duty cycle given Tmin_on = 600.00 ns is D = 0.006
Desired duty cycle given Vg = 1.00 kV and is Vout = 12.00 V is D = 0.012
Minimum output voltage Vout_min = 4.81 V
